## Exercício 1: Vestibular

Considere que a os dados gerados na célula abaixo contêm o número de acertos de 100 alunos em um vestibular para um curso de exatas, divididas pelos respectivos assuntos. Considere que cada assunto possui um número de questões conforme a tabela abaixo:

| assunto | número de questões |
|:---:|:---:|
| Matemática | 24 |
| Português | 18 |
| Geografia | 8 |
| Inglês | 8 |
| História | 8 |
| Física | 12 |
| Química | 12 |

Usando os comandos de operações com DataFrames que você aprendeu na Aula 03, calcule:

1. (operações com escalar) Calcule o percentual de acerto dos alunos por assunto.  
2. (operações entre *DataFrames) Calcule o total de acertos de cada aluno.  
3. Calcule o percentual geral de cada aluno.  
4. Suponha que a nota de corte para a segunda fase seja 45. Quantos alunos tiveram nota maior que 45?  

In [40]:
import numpy as np
import pandas as pd

# seed e número de alunos
np.random.seed(42)
n = 100

# máximos por disciplina (usei a tabela que você confirmou)
max_q = {
    'Matematica': 24,
    'Portugues' : 18,
    'Geografia' : 8,
    'Ingles'    : 8,
    'Historia'  : 8,
    'Fisica'    : 12,
    'Quimica'   : 12
}

# gerar DataFrame com acertos aleatórios (0..max inclusive)
df_all = pd.DataFrame({
    'Matematica': np.random.randint(0, max_q['Matematica'] + 1, size=n),
    'Portugues' : np.random.randint(0, max_q['Portugues'] + 1, size=n),
    'Geografia' : np.random.randint(0, max_q['Geografia'] + 1, size=n),
    'Ingles'    : np.random.randint(0, max_q['Ingles'] + 1, size=n),
    'Historia'  : np.random.randint(0, max_q['Historia'] + 1, size=n),
    'Fisica'    : np.random.randint(0, max_q['Fisica'] + 1, size=n),
    'Quimica'   : np.random.randint(0, max_q['Quimica'] + 1, size=n)
})

# exemplo rápido: mostrar primeiras linhas
print(df_all.head())


   Matematica  Portugues  Geografia  Ingles  Historia  Fisica  Quimica
0           6          6          6       3         5       8        8
1          19          8          3       2         8       6        1
2          14          0          6       0         4      12        8
3          10         11          7       0         0       4        0
4           7          7          0       4         3       9        0


In [59]:
# 1) Seu código aqui
for col, mx in max_q.items():
    df_all[f'{col}_pct'] = (df_all[col] / mx) * 100


In [60]:
# 2) Seu código aqui

df_all['Total_acertos'] = df_all[list(max_q.keys())].sum(axis=1)


In [61]:
# 3) Seu código aqui
total_questoes = sum(max_q.values())  # soma das questões de todas as disciplinas
df_all['Percentual_geral'] = (df_all['Total_acertos'] / total_questoes) * 100



In [62]:
# 4) Seu código aqui
n_maior_45 = (df_all['Total_acertos'] > 45).sum()



In [63]:
print("\nResumo dos totais (describe):")
print(df_all['Total_acertos'].describe())



Resumo dos totais (describe):
count    100.000000
mean      44.120000
std       10.530648
min       16.000000
25%       37.750000
50%       43.000000
75%       51.000000
max       66.000000
Name: Total_acertos, dtype: float64


## 2) Vestibular II

Ainda sobre o mesmo banco de dados:

1. Neste vestibular, quem 'zera' em matemática, física ou química está desqualificado. Monte um novo *DataFrame* com os alunos desqualificados por este critério.
2. Quantos são esses alunos?
3. Qual a média desses alunos em história e geografia?
4. Monte um *DataFrame* com os alunos que passaram para a segunda fase. Repare que estes alunos não podem ter sido desqualificados.

In [68]:
# seu código aqui
mask_desqual = (df_all['Matematica'] == 0) | (df_all['Fisica'] == 0) | (df_all['Quimica'] == 0)
df_desqualificados = df_all[mask_desqual].copy()
n_desqual = df_desqualificados.shape[0]
if n_desqual > 0:
    media_historia_desq = df_desqualificados['Historia'].mean()
    media_geografia_desq = df_desqualificados['Geografia'].mean()
else:
    media_historia_desq = float('nan')
    media_geografia_desq = float('nan')
mask_passaram = (df_all['Total_acertos'] > 45) & (~mask_desqual)
df_passaram = df_all[mask_passaram].copy()
n_passaram = df_passaram.shape[0]



## 3) Vacinações no Acre
Vamos trabalhar agora com a base de vacinações no Acre. Para facilitar a sua vida, copiamos o link do arquivo na célula abaixo.

1. Quantas vacinas estão registradas nessa base?  
2. Quantos pacientes foram vacinados? (considere um paciente para cada valor único de ```paciente_id```)  
3. Quantos pacientes únicos tomaram a primeira dose? OBS: Há um caractere especial neste campo. Receba os valores do campo com o método ```.unique()```.   
4. Quantos pacientes com menos de 18 anos foram vacinados?  
5. Quantos estabelecimentos aplicaram vacina no Acre?


**OBS:** O portal do DATASUS pode apresentar instabilidades, retornando um erro na segunda célula abaixo. Por este motivo está disponível uma base estática, que se for baixada para o seu *working directory* pode ser lida com este comando: ```df = pd.read_csv('registros de vacinacao covid ACRE.csv', sep=';')```.

**OBS2:** Para saber qual é o seu working directory, rode no jupyter: ```!pwd```.            

In [152]:
arquivo = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIPNI/COVID/uf/uf%3DAC/part-00000-5ed92752-6121-474f-ab37-816918134afc.c000.csv'

In [168]:
import pandas as pd
import io
from pathlib import Path
from datetime import datetime

# caminho exato do seu arquivo
p = Path(r"C:\Users\wilma\OneDrive\Área de Trabalho\projeto phyton\registros de vacinacao covid ACRE.csv")
if not p.exists():
    raise SystemExit(f"Arquivo não encontrado: {p}")

# detectar separador nas primeiras linhas
with p.open("r", encoding="utf-8", errors="ignore") as f:
    sample = "".join([next(f) for _ in range(5)])
sep = None
for s in [';',',','\t','|']:
    try:
        cols = pd.read_csv(io.StringIO(sample), sep=s, nrows=0).columns.tolist()
        if len(cols) > 1:
            sep = s
            break
    except Exception:
        continue
if sep is None:
    sep = ';'

# ler o arquivo (df_vac) com o separador detectado
with p.open("r", encoding="utf-8", errors="ignore") as f:
    txt = f.read()
df_vac = pd.read_csv(io.StringIO(txt), sep=sep, dtype=str, low_memory=False)
df_vac.columns = [c.strip() for c in df_vac.columns]

# helpers
def clean(s):
    return s.astype(str).str.strip().replace({"": pd.NA, "nan": pd.NA, "null": pd.NA, "None": pd.NA}).dropna()

# detectar colunas (baseado nas colunas exibidas anteriormente)
col_id = next((c for c in df_vac.columns if c.lower() in ('paciente_id','patient_id','document_id','documentid')), None)
col_birth = next((c for c in df_vac.columns if 'datanasc' in c.lower() or 'datanascimento' in c.lower() or 'data_nasc' in c.lower()), None)
col_dose = next((c for c in df_vac.columns if 'descricao' in c.lower() and 'dose' in c.lower()), None)
if not col_dose:
    col_dose = next((c for c in df_vac.columns if 'dose' in c.lower()), None)
col_observacao = next((c for c in df_vac.columns if 'observ' in c.lower() or 'estado_observacao' in c.lower() or 'em_observacao' in c.lower()), None)
col_estab = next((c for c in df_vac.columns if 'estabelecimento' in c.lower() or 'estalecimento' in c.lower() or 'unidade' in c.lower()), None)



In [170]:
# 1) Sua solução aqui
total_vacinas = len(df_vac)


In [171]:
# 2)
pacientes_unicos = int(clean(df_vac[col_id]).nunique()) if col_id else "coluna paciente_id não detectada"


In [172]:
# 3)
registros_primeira_dose = None
pacientes_primeira_unicos = None
if col_dose:
    dose_s = df_vac[col_dose].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii').str.lower()
    mask_1 = dose_s.str.contains(r'\b1\b|\b1a\b|1ª|primeira|primeiro|dose 1|dose1', na=False)
    registros_primeira_dose = int(mask_1.sum())
    if col_id:
        pacientes_primeira_unicos = int(clean(df_vac.loc[mask_1, col_id]).nunique())



In [173]:
# 4)
pacientes_menor18_unicos = None
menores_sample = None
if col_birth and col_id:
    nasc = pd.to_datetime(df_vac[col_birth].astype(str).str.strip(), dayfirst=False, errors='coerce', format="%Y-%m-%d")
    hoje = pd.Timestamp.now().normalize()
    idade = (hoje - nasc).dt.days // 365
    mask_menor = idade < 18
    pacientes_menor18_unicos = int(clean(df_vac.loc[mask_menor, col_id]).nunique())
    menores_sample = df_vac.loc[mask_menor, [col_id, col_birth]].drop_duplicates().head(20)




In [174]:
# 5)
observacao_unicos = None
if col_observacao and col_id:
    # considera valores truthy/indicadores de observação (ajuste conforme seus dados)
    obs_s = df_vac[col_observacao].astype(str).str.strip().str.lower()
    mask_obs = obs_s.isin(['sim','s','true','1','em observacao','em observação']) | obs_s.str.contains('observ', na=False)
    observacao_unicos = int(clean(df_vac.loc[mask_obs, col_id]).nunique())



## 4) Vacinação II
Gere um *DataFrame* que contenha somente os estabelecimentos que aplicaram vcinas a menores de 18 anos. Nesse *DataFrame* devem conter somente os dados dos estabelecimentos, mais uma coluna sendo a quantidade de vacinas que o estabelecimento aplicou a menores de 18 anos.  
  
1. crie uma cópia do *DataFrame* original, contendo somente os registros de vacinas realizadas a menores de 18 anos.  
2. crie uma lista das colunas desse *DataFrame* com o atributo de *DataFrame* **.columns()**  
3. Nesse *DataFrame* faça uma contagem do campo ```vacina_categoria_nome```.
3. a partir da lista de colunas, escolha somente aquelas que são referentes ao estabelecimento, faça uma lista com esses valores.  
4. usando o método *.loc*, selecione somente essas variáveis  
5. Aplique o método **.drop_duplicates** e crie uma lista com uma linha para cada estabelecimento, com os dados do estabelecimento  

In [181]:
# 1)
if not col_birth:
    raise SystemExit(f"Coluna de nascimento não detectada. Colunas disponíveis: {df.columns.tolist()}")
nasc = pd.to_datetime(df[col_birth].astype(str).str.strip(), dayfirst=False, errors='coerce', format="%Y-%m-%d")
hoje = pd.Timestamp.now().normalize()
idade_anos = (hoje - nasc).dt.days // 365
mask_menor18 = idade_anos < 18
df_menores18 = df.loc[mask_menor18].copy()


In [196]:
# 2)
colunas_df_menores18 = df_menores18.columns.tolist()


In [197]:
# 3)
if col_vac_cat and col_vac_cat in df_menores18.columns:
    contagem_vacina_categoria = df_menores18[col_vac_cat].astype(str).str.strip().value_counts()


In [199]:
# 4)
# seleciona_colunas_estabelecimento.py
import pandas as pd

if 'df_menores18' not in globals():
    raise SystemExit("df_menores18 não encontrado. Rode a célula que filtra menores de 18 antes.")

cols = df_menores18.columns.tolist()

# palavras-chave úteis para detectar colunas de estabelecimento
keywords = [
    'estabelecimento', 'estab', 'unidade', 'cnes', 'razao', 'razao_social', 'razaosocial',
    'fantasia', 'nofantasia', 'nome_estab', 'nome', 'endereco', 'municipio', 'município', 'cidade',
    'bairro', 'cep', 'uf', 'codigo_estabelecimento', 'codigocnes', 'codigo_cnes', 'cnes_codigo'
]

# normalizar nomes para comparação
cols_norm = [c.lower().strip() for c in cols]

# heurística: marcar colunas que contenham qualquer keyword
estab_mask = [any(k in cn for k in keywords) for cn in cols_norm]
estab_cols = [c for c, m in zip(cols, estab_mask) if m]

# ranking simples de candidatos (quantas keywords cada coluna contém)
def score_col(name):
    ln = name.lower()
    return sum(1 for k in keywords if k in ln)

ranked = sorted([(c, score_col(c)) for c in cols], key=lambda x: (-x[1], x[0]))
estab_cols_sugeridas = [c for c, s in ranked if s>0]



In [203]:
# 5)
# defina a lista corretamente (remova o # antes do colchete)
estab_cols = [
    'estabelecimento_valor',
    'estabelecimento_nome_fantasia',
    'estabelecimento_codigo',
    'estabelecimento_tipo',
    'estabelecimento_municipio_nome'
]  # ajuste para os nomes reais do seu dataset

# filtrar apenas as colunas de estabelecimento
estab_only = df_menores18.loc[:, [c for c in estab_cols if c in df_menores18.columns]]

# verificação rápida
print("Colunas selecionadas:", estab_only.columns.tolist())
print("Linhas antes de filtrar:", len(estab_only))

Colunas selecionadas: ['estabelecimento_valor', 'estabelecimento_municipio_nome']
Linhas antes de filtrar: 8


In [205]:
# 6)
estab_only = estab_only.copy()
estab_only['_estab_key'] = estab_only.fillna('').astype(str).agg('||'.join, axis=1)
contagem = estab_only.groupby('_estab_key').size().reset_index(name='vacinas_menores18')

# pegar uma linha representativa de dados do estabelecimento (primeira ocorrência)
dados_estab = estab_only.groupby('_estab_key', as_index=False).first()

# juntar dados representativos com a contagem e remover coluna auxiliar
estabelecimentos_menores18 = dados_estab.merge(contagem, on='_estab_key').drop(columns=['_estab_key'])

# ordenar por contagem decrescente e resetar índice
estabelecimentos_menores18 = estabelecimentos_menores18.sort_values(by='vacinas_menores18', ascending=False).reset_index(drop=True)

# verificação rápida
print("Estabelecimentos únicos (linhas):", len(estabelecimentos_menores18))
print("Exemplo (até 10):")
print(estabelecimentos_menores18.head(10).to_csv(index=False))



Estabelecimentos únicos (linhas): 8
Exemplo (até 10):
estabelecimento_valor,estabelecimento_municipio_nome,vacinas_menores18
2000040,FEIJO,1
3323307,PORTO ACRE,1
6428940,ASSIS BRASIL,1
6430201,JORDAO,1
6697151,SENA MADUREIRA,1
6748759,FEIJO,1
6917291,RIO BRANCO,1
7625855,MARECHAL THAUMATURGO,1

